In [1]:
# https://towardsdatascience.com/modeling-customer-lifetime-value-with-lifetimes-71171a35f654

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pandas_gbq
import os
pd.options.display.max_columns = 100
pd.options.display.max_rows = 300

import pandas as pd
from google.oauth2 import service_account

from scipy.stats import norm
import matplotlib.pyplot as plt
import scipy

from tqdm.auto import tqdm

#plt.style.use('ggplot')

import plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

import seaborn as sns
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [29]:
#df = pd.read_csv('data.csv')

- load date

In [ ]:
from lifetimes.datasets import load_dataset

transactions = load_dataset(
    filename='CDNOW_sample.txt', 
    header=None, 
    delim_whitespace=True, 
    names=['customer_id', 'customer_index', 'date', 'quantity', 'amount'],
    converters={'date': lambda x: pd.to_datetime(x, format="%Y%m%d")}
)

In [30]:
transactions.head(3)

,customer_id,customer_index,date,quantity,amount
0,4,1,1997-01-01,2,29.33
1,4,1,1997-01-18,2,29.73
2,4,1,1997-08-02,1,14.96


- restructure table to RFM format

In [31]:
from lifetimes.utils import summary_data_from_transaction_data

rfm = summary_data_from_transaction_data(transactions=transactions,
                                         customer_id_col='customer_id',
                                         datetime_col='date',
                                         monetary_value_col = 'amount',
                                         observation_period_end=pd.to_datetime('2022-07-28'),
                                         freq='W')

In [32]:
rfm

,frequency,recency,T,monetary_value
customer_id,,,,
4,3.0,49.0,1334.0,23.723333
18,0.0,0.0,1334.0,0.000000
21,1.0,2.0,1334.0,11.770000
50,0.0,0.0,1334.0,0.000000
60,0.0,0.0,1330.0,0.000000
...,...,...,...,...
23537,2.0,53.0,1322.0,19.775000
23551,5.0,24.0,1322.0,44.928000
23554,1.0,44.0,1322.0,24.600000


- Holdout split

In [34]:
from lifetimes.utils import calibration_and_holdout_data

rfm_cal_holdout = calibration_and_holdout_data(transactions=transactions,
                                               customer_id_col='customer_id', 
                                               datetime_col='date',
                                               monetary_value_col = 'amount',
                                               freq='W',
                                               calibration_period_end='1998-01-01',
                                               observation_period_end='1998-06-30' )

rfm_cal_holdout.head(3)

,frequency_cal,recency_cal,T_cal,monetary_value_cal,frequency_holdout,monetary_value_holdout,duration_holdout
customer_id,,,,,,,
4,3.0,49.0,52.0,23.723333,0.0,0.0,26.0
18,0.0,0.0,52.0,0.000000,0.0,0.0,26.0
21,1.0,2.0,52.0,11.770000,0.0,0.0,26.0


- Fitting

In [35]:
from lifetimes import BetaGeoFitter

# instantiation of BG-NBD model
bgf = BetaGeoFitter(penalizer_coef=0.0)

# fitting of BG-NBD model
bgf.fit(frequency=rfm_cal_holdout['frequency_cal'], 
        recency=rfm_cal_holdout['recency_cal'], 
        T=rfm_cal_holdout['T_cal'])

<lifetimes.BetaGeoFitter: fitted with 2357 subjects, a: 0.53, alpha: 6.73, b: 1.97, r: 0.28>

In [37]:
bgf.summary

,coef,se(coef),lower 95% bound,upper 95% bound
r,0.281847,0.014034,0.254340,0.309354
alpha,6.732810,0.532304,5.689494,7.776126
a,0.532360,0.122632,0.292000,0.772719
b,1.971260,0.575322,0.843628,3.098892
